In [ ]:
!pip install langchain-google-genai
!pip install langchain

In [ ]:
import pandas as pd
import numpy as np 

# Data Preprocessing

In [ ]:
#data loading
df=pd.read_csv("indicators.csv")

In [ ]:
df.sample(10)

# Data generation

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
response_schemas = [
    ResponseSchema(name="Instruction", description="Instruction or Question that user Might ask to get that script"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
template="""
You are Given the Detailed Summary of the Trading Indicators Scripts Create Instruction or Question that could be used to Generate that Script (provide only one Instruction)\n
\n{format_instructions}\nSummary: {context} ")

"""

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=template,
    input_variables=["context"],
    partial_variables={"format_instructions": format_instructions},
)

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ['GOOGLE_API_KEY'] = 'here is you gmini api key '
llm = ChatGoogleGenerativeAI(model="gemini-pro")
chain = prompt | llm | output_parser

In [ ]:
Description=list(df["Descriptions"])
code=list(df["code"])

In [ ]:
#Batches
# batch1=Description[0:3000]
# code_b1=code[0:3000]

batch2=Description[3000:6000]
code_b2=code[3000:6000]

# batch3=Description[6000:9000]
# code_b3=code[6000:9000]

In [ ]:
# instructions1=[]
instructions2=[]
progress_flag=0
# instructions3=[]

In [ ]:
from tqdm import tqdm
for i in tqdm(range(progress_flag,3000)):
    response=chain.invoke({"context":batch2[i]})
    instructions2.append(response["Instruction"])
    progress_flag=progress_flag+1
    

In [ ]:
final_data=pd.DataFrame(columns=["Instructions","code"])

In [ ]:
final_data["Instructions"]=instructions2
final_data["code"]=code_b2

In [ ]:
final_data

In [ ]:
final_data.to_csv("pinescript_indicator_instructions_2.csv",index=False)
